<a href="https://colab.research.google.com/github/ntorenduwayo/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|     223980852|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|   N|                Y|Good quality. Shi...|Exactly as descri...| 2015-08-31|
|         US|   18125776|R3QWMLJHIW6P37|B0000537JQ|     819771537|     Ezy Dose Weekly|Personal_Care_App

In [4]:
# Read in the vine_table.csv file as a DataFrame
## Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1QX6706ZWJ1P5|          5|            0|          0|   N|                Y|
|R3QWMLJHIW6P37|          5|            0|          0|   N|                Y|
|R14Z1VR1N0Z9G6|          5|            1|          1|   N|                Y|
| R25ZRJL0GH0U0|          2|            0|          0|   N|                Y|
|R3837KYH7AZNIY|          4|            0|          1|   N|                Y|
|R2MN0QYCY6EVIV|          5|            0|          0|   N|                Y|
|R3AN2UJ1D42ZS0|          5|            0|          0|   N|                Y|
|R3U29ZLUWEEK4M|          5|            0|          0|   N|                Y|
|R16ZDMJJHK796C|          5|            0|          0|   N|                N|
| RRRDOEJZD1Y22|          4|            0|          0|   N|     

In [5]:
# 1. Filter the data and create a new DataFrame to retrieve all the rows where the total_votes count is  above or equal to 20
df_tvotes = vine_df.filter(df.total_votes >= 20)
df_tvotes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2PT2X8FMYFCV3|          5|           39|         41|   N|                Y|
| RGCUQSH80SYY7|          1|           36|         47|   N|                N|
|R3E6EI073KP0AL|          2|           21|         36|   N|                Y|
|R3MYTXW7B4Z6YW|          4|          687|        711|   N|                Y|
|R2WMF1IWN5ZYO4|          5|           18|         24|   N|                Y|
| ROX6L7VJFS2P7|          5|          339|        371|   N|                N|
| RRKAST7D4QTW8|          1|           20|         27|   N|                N|
|R2QBKTWBDPL8BF|          5|           65|         83|   N|                N|
|R3ULT9I0U79N8L|          5|           49|         57|   N|                N|
|R20LABVEB8O83Q|          3|           11|         23|   N|     

In [6]:
# 2. Filter the new DataFrame created in Step 1 and create a new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
df_helpful_votes = df_tvotes.filter((df_tvotes.helpful_votes / df_tvotes.total_votes) >= 0.5)
df_helpful_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2PT2X8FMYFCV3|          5|           39|         41|   N|                Y|
| RGCUQSH80SYY7|          1|           36|         47|   N|                N|
|R3E6EI073KP0AL|          2|           21|         36|   N|                Y|
|R3MYTXW7B4Z6YW|          4|          687|        711|   N|                Y|
|R2WMF1IWN5ZYO4|          5|           18|         24|   N|                Y|
| ROX6L7VJFS2P7|          5|          339|        371|   N|                N|
| RRKAST7D4QTW8|          1|           20|         27|   N|                N|
|R2QBKTWBDPL8BF|          5|           65|         83|   N|                N|
|R3ULT9I0U79N8L|          5|           49|         57|   N|                N|
| RAC19YVDYSIKB|          5|          161|        178|   N|     

In [7]:
# 3. Create paid vine DataFrame
## Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y' ##
df_paidvine = df_helpful_votes.filter(df_helpful_votes.vine == 'Y')
df_paidvine.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3R2RHQWEKX5NL|          5|           81|         89|   Y|                N|
|R1RZ4JZO8DAP7H|          3|          200|        227|   Y|                N|
| RQ94HI0WM8KIG|          5|          174|        188|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+



In [8]:
# 4. Create unpaid vine DataFrame
df_unpaidvine = df_helpful_votes.filter(df_helpful_votes.vine == 'N')
df_unpaidvine.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2PT2X8FMYFCV3|          5|           39|         41|   N|                Y|
| RGCUQSH80SYY7|          1|           36|         47|   N|                N|
|R3E6EI073KP0AL|          2|           21|         36|   N|                Y|
|R3MYTXW7B4Z6YW|          4|          687|        711|   N|                Y|
|R2WMF1IWN5ZYO4|          5|           18|         24|   N|                Y|
| ROX6L7VJFS2P7|          5|          339|        371|   N|                N|
| RRKAST7D4QTW8|          1|           20|         27|   N|                N|
|R2QBKTWBDPL8BF|          5|           65|         83|   N|                N|
|R3ULT9I0U79N8L|          5|           49|         57|   N|                N|
| RAC19YVDYSIKB|          5|          161|        178|   N|     

In [9]:
# 5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid)
## Paid total number of reviews
total_paid_reviews = df_paidvine.count()
total_paid_reviews

3

In [10]:
## Paid 5-star reviews
paid_five_star_reviews = df_paidvine.filter(df_paidvine.star_rating == 5).count()
paid_five_star_reviews

2

In [11]:
## Paid 5-star reviews percentage
paid_five_star_reviews_percent = (paid_five_star_reviews/total_paid_reviews) * 100
paid_five_star_reviews_percent

66.66666666666666

In [12]:
## Unpaid total number of reviews
total_unpaid_reviews = df_unpaidvine.count()
total_unpaid_reviews

3094

In [13]:
## Unpaid 5-star reviews
unpaid_five_star_reviews = df_unpaidvine.filter(df_unpaidvine.star_rating == 5).count()
unpaid_five_star_reviews

1704

In [14]:
## Unpaid 5-star reviews percentage
unpaid_five_star_reviews_percent = (unpaid_five_star_reviews/total_unpaid_reviews) * 100
unpaid_five_star_reviews_percent

55.07433742727861